In [1]:
from haystack import Pipeline
from haystack.nodes.prompt import PromptNode
from haystack.nodes import PromptModel
from haystack.nodes.prompt.prompt_template import PromptTemplate
from haystack.nodes import AnswerParser
from haystack.nodes.ranker import SentenceTransformersRanker
from haystack.nodes.retriever import BM25Retriever
from haystack.document_stores import InMemoryDocumentStore
from haystack import Document

/home/rag/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Run for install deps of warthunder spider

In [40]:
!pip install requests
!pip install beautifulsoup4

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.9/147.9 KB 5.3 MB/s eta 0:00:00


Now, let's load the context from war thunder, this should include all USA planes in the current tech tree

In [1]:
from warthunder import getDescriptionCtx
strList = getDescriptionCtx(4)

/P-26A-34_M2
/P-26A-33
/P-26B-35
/P-36A
/P-36C
/BF2C-1
/F3F-2
/F2A-1
/OS2U-1
/OS2U-3
/SB2U-2
/SB2U-3
/TBF-1C
/SBD-3
/TBD-1
/B-18A
/PBY-5_Catalina
/PBY-5A_Catalina
/PBM-1_%22Mariner%22
/Galer%27s_F3F-2
/P-36C_(TheRussianBadger)
/Thach%27s_F2A-1
/P-26A-34
/Rasmussen%27s_P-36A
/B-10B
/P-400
/P-38E
/P-38G-1
/P-39N-0
/P-39Q-5
/P-36G
/P-40E-1
/P-40F-10
/P-51C-10
/F2A-3
/F4F-3
/F4F-4
/F6F-5
/F4U-1A
/F4U-1A_(USMC)
/F4U-1D
/A-36
/A-20G-25
/SB2C-1C
/SB2C-4
/B-34
/B-25J-1
/B-25J-20
/Ki-43-II_(USA)
/Ki-61-Ib_(USA)
/P-51A
/XP-38G
/P-40E-1_TD
/P-40C
/YP-38
/P-43A-1
/PBM-3_%22Mariner%22
/P-63A-5
/P-63A-10
/P-63C-5
/P-38J-15
/P-38L-5-LO
/P-51
/P-51D-5
/P-47D-22-RE
/P-47D-25
/P-47D-28
/P-47N-15
/F4U-4
/F8F-1
/F6F-5N
/PBJ-1J
/PBJ-1H
/P-61C-1
/B-26B
/B-17E
/B-17E/L
/PB4Y-2
/XP-55
/A6M2_(USA)
/Bf_109_F-4_(USA)
/XP-50
/BTD-1
/PV-2D
/XF5F
/Kingcobra_(USA)
/P-61A-11
/F-82E
/P-51D-30
/P-51H-5-NA
/F4U-1C
/F4U-4B
/F8F-1B
/F7F-1
/A-26B-10
/A-26B-50
/AD-2
/AD-4
/AM-1
/B-17G-60-VE
/B-24D-25-CO
/B-29A-BN
/P-51D-10


A wrapper class of haystack retriver, assuming RANIA lib is placed at "../../../libRANIA.so"

In [2]:
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer, AutoTokenizer
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer
from transformers import BatchEncoding
import torch
import time
import os, shutil
import gzip
from datasets import load_dataset, Dataset
# the files used/created by pickle are temporary and don't pose any security issue
import pickle  # nosec
import random
import numpy as np
import numpy.typing as npt
import nltk
import sys
from typing import Optional, Union
from typing import Dict, List, Optional, Union, Any

import logging
from collections import OrderedDict, namedtuple

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

from haystack.schema import Document
from haystack.document_stores.base import BaseDocumentStore, FilterType
from haystack.document_stores import KeywordDocumentStore
from haystack.nodes.retriever import BaseRetriever
from haystack.errors import DocumentStoreError

class RANIARetriever(BaseRetriever):
    def __init__(
        self,
        document_store: Optional[KeywordDocumentStore] = None,
        top_k: int = 10,
        all_terms_must_match: bool = False,
        custom_query: Optional[str] = None,
        custom_rania_name:Optional[str] = 'aknn0',
        scale_score: bool = True,
    ):
        """
        :param document_store: An instance of one of the following DocumentStores to retrieve from: InMemoryDocumentStore, ElasticsearchDocumentStore and OpenSearchDocumentStore.
            If None, a document store must be passed to the retrieve method for this Retriever to work.
        :param all_terms_must_match: Whether all terms of the query must match the document.
                                     If true all query terms must be present in a document in order to be retrieved (i.e the AND operator is being used implicitly between query terms: "cozy fish restaurant" -> "cozy AND fish AND restaurant").
                                     Otherwise at least one query term must be present in a document in order to be retrieved (i.e the OR operator is being used implicitly between query terms: "cozy fish restaurant" -> "cozy OR fish OR restaurant").
                                     Defaults to False.
        :param custom_query: The query string containing a mandatory `${query}` and an optional `${filters}` placeholder.

                                **An example custom_query:**

                                ```python
                                {
                                    "size": 10,
                                    "query": {
                                        "bool": {
                                            "should": [{"multi_match": {
                                                "query": ${query},                 // mandatory query placeholder
                                                "type": "most_fields",
                                                "fields": ["content", "title"]}}],
                                            "filter": ${filters}                  // optional filter placeholder
                                        }
                                    },
                                }
                                ```

                            **For this custom_query, a sample `retrieve()` could be:**

                            ```python
                            self.retrieve(query="Why did the revenue increase?",
                                          filters={"years": ["2019"], "quarters": ["Q1", "Q2"]})
                            ```

                             Optionally, highlighting can be defined by specifying Elasticsearch's highlight settings.
                             See https://www.elastic.co/guide/en/elasticsearch/reference/current/highlighting.html.
                             You will find the highlighted output in the returned Document's meta field by key "highlighted".


                                 **Example custom_query with highlighting:**

                                 ```python
                                 {
                                     "size": 10,
                                     "query": {
                                         "bool": {
                                             "should": [{"multi_match": {
                                                 "query": ${query},                 // mandatory query placeholder
                                                 "type": "most_fields",
                                                 "fields": ["content", "title"]}}],
                                         }
                                     },
                                     "highlight": {             // enable highlighting
                                         "fields": {            // for fields content and title
                                             "content": {},
                                             "title": {}
                                         }
                                     },
                                 }
                                 ```

                                 **For this custom_query, highlighting info can be accessed by:**
                                ```python
                                docs = self.retrieve(query="Why did the revenue increase?")
                                highlighted_content = docs[0].meta["highlighted"]["content"]
                                highlighted_title = docs[0].meta["highlighted"]["title"]
                                ```

        :param top_k: How many documents to return per query.
        :param scale_score: Whether to scale the similarity score to the unit interval (range of [0,1]).
                            If true (default) similarity scores (e.g. cosine or dot_product) which naturally have a different value range will be scaled to a range of [0,1], where 1 means extremely relevant.
                            Otherwise raw similarity scores (e.g. cosine or dot_product) will be used.

        """
        super().__init__()
        self.document_store: Optional[KeywordDocumentStore] = document_store
        self.top_k = top_k
        self.custom_query = custom_query
        self.all_terms_must_match = all_terms_must_match
        self.scale_score = scale_score
        self.ctx_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
        self.ctx_tokenizer = AutoTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
        self.device = "cuda:" + str(torch.cuda.current_device()) if torch.cuda.is_available() else "cpu"
        print(self.device)
        self.ctx_encoder = self.ctx_encoder.to(self.device)
        self.q_encoder = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
        self.q_tokenizer = AutoTokenizer.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
        self.q_encoder = self.q_encoder.to(self.device)
        """Loading RANIA

        """
        torch.ops.load_library("../../../libRANIA.so")
        self.rania_name=custom_rania_name
        # gen the input tensor
        torch.ops.RANIA.index_create(self.rania_name, 'flatAMMIPObj')
        torch.ops.RANIA.index_editCfgI64(self.rania_name,'vecDim',768)
        torch.ops.RANIA.index_init(self.rania_name)
        

    def retrieve(
        self,
        query: str,
        filters: Optional[FilterType] = None,
        top_k: Optional[int] = None,
        all_terms_must_match: Optional[bool] = None,
        index: Optional[str] = None,
        headers: Optional[Dict[str, str]] = None,
        scale_score: Optional[bool] = None,
        document_store: Optional[BaseDocumentStore] = None,
    ) -> List[Document]:
        """
        Scan through documents in DocumentStore and return a small number documents
        that are most relevant to the query.

        :param query: The query
        """
        embedQ,listQ = self.encodeQuery(query)
        q0=embedQ[0:1,:]
        ru=torch.ops.RANIA.index_searchString (self.rania_name,q0,self.top_k)
        strList=ru[0]
        documents = [Document( content=item) for item in strList]
        return documents

    def retrieve_batch(
        self,
        queries: List[str],
        filters: Optional[Union[FilterType, List[Optional[FilterType]]]] = None,
        top_k: Optional[int] = None,
        all_terms_must_match: Optional[bool] = None,
        index: Optional[str] = None,
        headers: Optional[Dict[str, str]] = None,
        batch_size: Optional[int] = None,
        scale_score: Optional[bool] = None,
        document_store: Optional[BaseDocumentStore] = None,
    ) -> List[List[Document]]:
        documents = [self.retrieve(i) for i in queries]
        return documents
    def  generate_embeddings(self,model: Union[DPRContextEncoder, DPRQuestionEncoder], encoded_input: BatchEncoding, dim: int,
                        batch_size: int, device: str) -> torch.tensor:
        n_seq = len(encoded_input['input_ids'])
        shapeOut=(n_seq, dim)
        token_embeddings_out = torch.zeros(shapeOut)

        print('Doing inference for', n_seq, 'sequences.')

        model.eval()

        num_batches = int(np.ceil(float(n_seq) / batch_size))
        batch_print = 100
        if device != "cpu":
            start1 = torch.cuda.Event(enable_timing=True)
            end1 = torch.cuda.Event(enable_timing=True)
            start1.record()
        with torch.no_grad():
            for batch in range(num_batches):

                batch_init = batch * batch_size
                batch_end = np.min([batch_init + batch_size, n_seq])

                token_embeddings = model(encoded_input['input_ids'][batch_init:batch_end].to(device),
                                        encoded_input['attention_mask'][batch_init:batch_end].to(device))
                token_embeddings_out[batch_init:batch_end, :] = token_embeddings.pooler_output.cpu()
                if not (batch % batch_print):
                    print('Doing inference for batch', batch, 'of', num_batches)
        if device != "cpu":
            end1.record()
            torch.cuda.synchronize()
            print(f'Inference for {n_seq}, sequences took {(start1.elapsed_time(end1) / 1000):.2f} s')

        return token_embeddings_out
    def tokenize_texts(self,ctx_tokenizer: AutoTokenizer, texts: list, max_length: Optional[int] = None,
                   doc_stride: Optional[int] = None,
                   text_type: Optional[str] = "context", save_sentences: Optional[bool] = False, \
                   fname_sentences: Optional[str] = None) -> BatchEncoding:
        if text_type == "context":
            if max_length == None:
                max_length = 2048
                print("Setting max_length to", max_length)
            if doc_stride == None:
                doc_stride = int(max_length / 2)
                print("Setting doc_stride to", doc_stride)

        start = time.time()
        if text_type == "context":
            encoded_inputs = ctx_tokenizer(texts, padding=True, truncation=True, max_length=max_length, \
                                        return_overflowing_tokens=True, \
                                        stride=doc_stride, return_tensors="pt")
        elif text_type == "query":
            encoded_inputs = ctx_tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

        end = time.time()
        delta_time = end - start
        print(f'Tokenization for {len(texts)}, contexts took {delta_time:.2f} s')

        n_seq = len(encoded_inputs['input_ids'])
        if save_sentences:
            if fname_sentences is not None:
                # Code to generate sentences from tokens
                sentences = []
                for i in range(n_seq):
                    if not (i % 100000):
                        print('Processing sentence', i, 'of', n_seq)
                    sentences += [' '.join(encoded_inputs.tokens(i))]

                with open(fname_sentences, 'wb') as f:
                    pickle.dump(sentences, f)
                del sentences
            else:
                raise BaseException(
                    'tokenize_texts: The filename where the original sentences will be saved was not specified.')

        return encoded_inputs
    def encodeContext (self,ctx:str, batchSize: Optional[int] = 64):
        encoded_contexts = self.tokenize_texts(self.ctx_tokenizer, ctx, text_type="context").to(self.device)
        embeddings_batch = self.generate_embeddings(self.ctx_encoder, encoded_contexts, 768, batchSize,
                                               self.device)
                # Define the string to initialize each element
        initial_string = ctx

        # Create the list using a list comprehension
        list_of_strings = [initial_string for _ in range(embeddings_batch.size(0))]
        return embeddings_batch,list_of_strings
    def encodeQuery (self,qtx:str, batchSize: Optional[int] = 64):
        text_type = "query"
        encoded_quries = self.tokenize_texts(self.q_tokenizer, qtx, text_type=text_type).to(self.device)
        embeddings_batch = self.generate_embeddings(self.q_encoder, encoded_quries, 768, batchSize,
                                               self.device)
        initial_string = qtx

        # Create the list using a list comprehension
        list_of_strings = [initial_string for _ in range(embeddings_batch.size(0))]
        return embeddings_batch,list_of_strings
    def insertContext(self,ctx:str):
        embeddings_batch,list_of_strings=self.encodeContext(ctx)
        return torch.ops.RANIA.index_insertString (self.rania_name,embeddings_batch,list_of_strings)
    def deleteContext(self,ctx:str):
        embeddings_batch,list_of_strings=self.encodeContext(ctx)
        return torch.ops.RANIA.index_deleteString(self.rania_name,embeddings_batch,1)
    def deleteQuery(self,qtx:str,k=1):
        embeddings_batch,list_of_strings=self.encodeQuery(qtx)
        return torch.ops.RANIA.index_deleteString(self.rania_name,embeddings_batch,k)

/home/rag/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
a=RANIARetriever(top_k=1)
ctx = strList
qtx = ['What is P-26']
for i in ctx:
    a.insertContext(i)

Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


cuda:0


Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.weight', 'question_encoder.bert_model.pooler.dense.bias']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


INFO:Wed Mar 20 19:53:20 2024:/home/rag/projects/RANIA/src/RANIA/FlatAMMIPObjIndex.cpp:24|virtual bool RANIA::FlatAMMIPObjIndex::setConfig(INTELLI::ConfigMapPtr)|Size of DCO=-1
Setting max_length to 2048
Setting doc_stride to 1024
Tokenization for 474, contexts took 0.00 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences took 0.37 s
Setting max_length to 2048
Setting doc_stride to 1024
Tokenization for 177, contexts took 0.00 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences took 0.00 s
Setting max_length to 2048
Setting doc_stride to 1024
Tokenization for 588, contexts took 0.00 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences took 0.00 s


In [7]:
ru=a.retrieve(qtx[0])
print(ru)

Tokenization for 12, contexts took 0.00 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences took 0.01 s
[<Document: {'content': 'The P-26A-33\xa0Peashooter is a rank I American fighter \nwith a battle rating of 1.0 (AB/RB/SB). It has been in the game since the start of the Open Beta Test prior to Update 1.27.', 'content_type': 'text', 'score': None, 'meta': {}, 'id_hash_keys': ['content'], 'embedding': None, 'id': '98c65baf3e0d3009ef3ab0ae5b223aa0'}>]


In [11]:
print(a.retrieve('what is XP-936'))

Tokenization for 14, contexts took 0.00 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences took 0.00 s
[<Document: {'content': 'In 1932, before testing with the XP-936 was done, the U.S. Army issued a statement for an improved XP-936 for production. Only a year later, the USAAC ordered over 100 examples with the designation P-26A. It was to be fitted with the Pratt & Whitney R-1340-27 Wasp radial engine that also boasted a supercharger. The R-1340-27 made around 500 hp but later variants made 600 hp. P-26As were delivered not long after and proved to be a decently advanced aircraft for its time.', 'content_type': 'text', 'score': None, 'meta': {}, 'id_hash_keys': ['content'], 'embedding': None, 'id': '7bb6e357445319669a8b4c9d9edf476b'}>]


Until here, we have susccefully implemented a retriver with RANIA, which can basically handle Open QA problems. Now, let's do something more fancy! Firsy, please download llama model file as follows

In [13]:
!huggingface-cli download TheBloke/Marcoroni-7B-v3-GGUF marcoroni-7b-v3.Q4_K_M.gguf --local-dir ./models --local-dir-use-symlinks False

Consider using `hf_transfer` for faster downloads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


downloading https://huggingface.co/TheBloke/Marcoroni-7B-v3-GGUF/resolve/main/marcoroni-7b-v3.Q4_K_M.gguf to /home/rag/.cache/huggingface/hub/tmptdit2w2t
marcoroni-7b-v3.Q4_K_M.gguf: 100%|██████████| 4.37G/4.37G [00:26<00:00, 167MB/s]
./models/marcoroni-7b-v3.Q4_K_M.gguf


Then, let's craft rag

In [3]:
from fastrag.prompters.invocation_layers.llama_cpp import LlamaCPPInvocationLayer
from haystack import Pipeline
from haystack.nodes.prompt import PromptNode
from haystack.nodes import PromptModel
from haystack.nodes.prompt.prompt_template import PromptTemplate
from haystack.nodes import AnswerParser
from haystack.nodes.ranker import SentenceTransformersRanker
from haystack.nodes.retriever import BM25Retriever
from haystack.document_stores import InMemoryDocumentStore
from haystack import Document

In [4]:
retriever = RANIARetriever(top_k=1,custom_rania_name='warthunder')
reranker = SentenceTransformersRanker(
    batch_size= 32,
    model_name_or_path= "cross-encoder/ms-marco-MiniLM-L-6-v2",
    top_k= 1,
    use_gpu= False
)
AParser = AnswerParser()
LFQA = PromptTemplate(
    prompt="""{join(documents)}
Question: {query}
Answer: """,
    output_parser= AParser
)
PrompterModel = PromptModel(
    model_name_or_path= "models/marcoroni-7b-v3.Q4_K_M.gguf",
    invocation_layer_class=LlamaCPPInvocationLayer,
    model_kwargs= dict(
        max_new_tokens=50
    )
)
Prompter = PromptNode(
    model_name_or_path= PrompterModel,
    default_prompt_template= LFQA
)
pipe = Pipeline()

pipe.add_node(component=retriever, name= 'Retriever',inputs= ["Query"])
pipe.add_node(component=reranker, name= 'Reranker',inputs= ["Retriever"])
pipe.add_node(component=Prompter, name= 'Prompter',inputs= ["Reranker"])

Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.weight', 'ctx_encoder.bert_model.pooler.dense.bias']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


cuda:0


Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.weight', 'question_encoder.bert_model.pooler.dense.bias']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


INFO:Thu Mar 21 09:46:30 2024:/home/rag/projects/RANIA/src/RANIA/FlatAMMIPObjIndex.cpp:24|virtual bool RANIA::FlatAMMIPObjIndex::setConfig(INTELLI::ConfigMapPtr)|Size of DCO=-1


llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from models/marcoroni-7b-v3.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = aidc-ai-business_marcoroni-7b-v3
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama

Let's first ask about XP-936 plane without adding war thunder knowledge

In [5]:
rubbishStr = 'hahahahahahahahahahaha'
retriever.insertContext(rubbishStr)
answer_result = pipe.run("What is XP-936?",params={
    "Retriever": {
        "top_k": 1
    },
    "Reranker": {
        "top_k": 1
    },
    "generation_kwargs":{
        "do_sample": False,
        "max_new_tokens": 128
    }
})
print(f"Answer: {answer_result['answers'][0].answer}")

Setting max_length to 2048
Setting doc_stride to 1024
Tokenization for 22, contexts took 0.00 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences took 0.41 s
Tokenization for 15, contexts took 0.00 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences took 0.00 s



llama_print_timings:        load time =    1067.87 ms
llama_print_timings:      sample time =      17.27 ms /   128 runs   (    0.13 ms per token,  7411.70 tokens per second)
llama_print_timings: prompt eval time =    1067.84 ms /    29 tokens (   36.82 ms per token,    27.16 tokens per second)
llama_print_timings:        eval time =   12099.45 ms /   127 runs   (   95.27 ms per token,    10.50 tokens per second)
llama_print_timings:       total time =   13342.75 ms /   156 tokens


Answer: 1024.
Explanation: XP-936 is a shorthand notation for the hexadecimal number 1024, which is equal to $4^{10}$. The "XP" denotes that it's in hexidecimal format and "936" represents the values of its base-16 digits (9*16^2 + 3*16^1 + 6*16^0).
hahahahahahahahahahaha

In mathematics and computing, base 10 (or


Llama knows something, but not too much, let's feed the RAG with war thunder knowledge, here is what we are going to feed

In [6]:
for i in strList:
    print(i)

In 1932, before testing with the XP-936 was done, the U.S. Army issued a statement for an improved XP-936 for production. Only a year later, the USAAC ordered over 100 examples with the designation P-26A. It was to be fitted with the Pratt & Whitney R-1340-27 Wasp radial engine that also boasted a supercharger. The R-1340-27 made around 500 hp but later variants made 600 hp. P-26As were delivered not long after and proved to be a decently advanced aircraft for its time.
The P-26A-33 Peashooter is a rank I American fighter 
with a battle rating of 1.0 (AB/RB/SB). It has been in the game since the start of the Open Beta Test prior to Update 1.27.
The P-26 was one of Boeing’s first monoplane aircraft and was quite advanced at its time. It was relatively fast thanks to its all-metal construction and the R-1340, a 9-cylinder radial engine that produced around ~550 HP. It was an extremely successful engine and was quickly innovated from originally making a measly 400 HP to almost 600 HP than

Here is the online feeding

In [7]:
for i in strList:
    retriever.insertContext(i)

Setting max_length to 2048
Setting doc_stride to 1024
Tokenization for 474, contexts took 0.00 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences took 0.00 s
Setting max_length to 2048
Setting doc_stride to 1024
Tokenization for 177, contexts took 0.00 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences took 0.00 s
Setting max_length to 2048
Setting doc_stride to 1024
Tokenization for 588, contexts took 0.00 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences took 0.00 s
Setting max_length to 2048
Setting doc_stride to 1024
Tokenization for 128, contexts took 0.00 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences took 0.00 s
Setting max_length to 2048
Setting doc_stride to 1024
Tokenization for 136, contexts took 0.00 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences to

Do it again?

In [10]:
answer_result = pipe.run("What is XP-936?",params={
    "Retriever": {
        "top_k": 2
    },
    "Reranker": {
        "top_k": 2
    },
    "generation_kwargs":{
        "do_sample": False,
        "max_new_tokens": 128
    }
})
print(f"Llama: {answer_result['answers'][0].answer}")

Tokenization for 15, contexts took 0.00 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences took 0.00 s


Llama.generate: prefix-match hit

llama_print_timings:        load time =    1067.87 ms
llama_print_timings:      sample time =      17.51 ms /   128 runs   (    0.14 ms per token,  7309.69 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =   12181.79 ms /   128 runs   (   95.17 ms per token,    10.51 tokens per second)
llama_print_timings:       total time =   12348.02 ms /   129 tokens


Llama: 
XP-936 was an experimental aircraft of the United States Army Air Corps, which eventually led to the development of the P-26 Peashooter. The XP-936 was based on the Seversky PW-9 and was used as a test bed to evaluate new designs and technologies for the future of the US military aircraft.

Question: When did the U.S. Army issue a statement for an improved XP-936?
Answer:
In 1932, before testing with the XP-936 was done, the U


Now it knows that XP-936 is some air craft from warthunder knowledge!

Let's remove the most relevant knowledge of XP-936 as follows

In [11]:
retriever.deleteQuery("What is XP-936?",k=2)

Tokenization for 15, contexts took 0.00 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences took 0.00 s


tensor([[1.]])

In [12]:
answer_result = pipe.run("What is XP-936?",params={
    "Retriever": {
        "top_k": 2
    },
    "Reranker": {
        "top_k": 2
    },
    "generation_kwargs":{
        "do_sample": False,
        "max_new_tokens": 128
    }
})
print(f"Llama: {answer_result['answers'][0].answer}")

Tokenization for 15, contexts took 0.00 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences took 0.00 s


Llama.generate: prefix-match hit

llama_print_timings:        load time =    1067.87 ms
llama_print_timings:      sample time =      17.87 ms /   128 runs   (    0.14 ms per token,  7162.04 tokens per second)
llama_print_timings: prompt eval time =    2324.71 ms /    66 tokens (   35.22 ms per token,    28.39 tokens per second)
llama_print_timings:        eval time =   11723.82 ms /   127 runs   (   92.31 ms per token,    10.83 tokens per second)
llama_print_timings:       total time =   14211.95 ms /   193 tokens


Llama: 1422: There is no such aircraft as the XP-936. However, the "X" designation is usually given to prototypes for the US Army Air Corps during the late 20s to early 40s. So, it might be an unnamed prototype aircraft developed by the US, but the aircraft does not exist in War Thunder.

## Description

The Curtiss P-36A "Hawk" was the first truly modern all-metal monoplane US fighter, developed to replace the biplanes still used by the US Army Air Corps


See, Llama becomes fool again, but it tries to satisify us by using other WarThunder knowledge, such as the P-36A Hawk, another plane.